In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scopus-5/ABCDD.csv


In [2]:

! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu pyspark==2.4.7
import nlu

debconf: delaying package configuration, since apt-utils is not installed
     |████████████████████████████████| 158 kB 1.2 MB/s 
     |████████████████████████████████| 217.9 MB 53 kB/s 
     |████████████████████████████████| 197 kB 35.7 MB/s 
     |████████████████████████████████| 139 kB 60.2 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.7-py2.py3-none-any.whl size=218279465 sha256=892ae59af526ce7edd4b12de623b5209c381be80e62bec9f9e4c0ada99d08dbc
  Stored in directory: /root/.cache/pip/wheels/da/28/74/56054e5fe3413c8c58b67e4d7483d4864fe483920c9b8ec754
Successfully built pyspark


In [3]:
nlu.load('emotion').predict('wow that was easy')


classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[OK!]
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]




,sentence_embeddings,sentence,emotion_confidence,emotion
origin_index,,,,
0,"[-0.01886986754834652, 0.028681324794888496, 0...",wow that was easy,0.999585,surprise


In [4]:

data = pd.read_csv("../input/scopus-5/ABCDD.csv")
df = data[['Abstract']]

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
df['Abstract'] = df['Abstract'].str.replace('[^a-zA-Z]',' ').str.lower()

In [7]:
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'

In [8]:
df['Abstract'] = df['Abstract'].str.replace(stop_re, '')


In [9]:
df['Abstract'].head()

0    several fundamental problems  arise  optimizat...
1    electronic voltage transformer   electricity p...
2    various methods  machine learning   implemente...
3      study   propose  model  consciousness    imp...
4    examination performance    measure  competence...
Name: Abstract, dtype: object

In [10]:
pipe = nlu.load('embed_sentence.bert')
predictions = pipe.predict(df.Abstract, output_level='document')
predictions

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


,Abstract,document,embed_sentence_bert_embeddings,text
origin_index,,,,
0,several fundamental problems arise optimizat...,several fundamental problems arise optimizatio...,"[-0.30958321690559387, 0.32803720235824585, -0...",several fundamental problems arise optimizat...
1,electronic voltage transformer electricity p...,electronic voltage transformer electricity par...,"[0.029165567830204964, 0.8990737795829773, -0....",electronic voltage transformer electricity p...
2,various methods machine learning implemente...,various methods machine learning implemented m...,"[-0.48946917057037354, 0.8942908644676208, -0....",various methods machine learning implemente...
3,study propose model consciousness imp...,study propose model consciousness implemented ...,"[-0.1395428478717804, 0.46414247155189514, -0....",study propose model consciousness imp...
4,examination performance measure competence...,examination performance measure competence exi...,"[-0.7468781471252441, 1.0565576553344727, -0.3...",examination performance measure competence...
5,paper discusses risk assessment changed ar...,paper discusses risk assessment changed artifi...,"[-0.43900567293167114, 1.5381633043289185, -0....",paper discusses risk assessment changed ar...
6,study big data machine learning offers sign...,study big data machine learning offers signifi...,"[-0.6192795634269714, 1.1968283653259277, -0.8...",study big data machine learning offers sign...
7,artificial intelligence finding way ever a...,artificial intelligence finding way ever appli...,"[-0.1351013481616974, 0.9480631351470947, -0.4...",artificial intelligence finding way ever a...
8,advent incorporation technology businesses...,advent incorporation technology businesses ref...,"[-0.15662378072738647, 1.1727793216705322, -0....",advent incorporation technology businesses...


In [11]:
from sklearn.metrics.pairwise import cosine_similarity


In [12]:
# put all sentence embeddings in a matrix
e_col = 'embed_sentence_bert_embeddings'

def get_sim_df_for_iloc (sent_iloc, predictions=predictions, e_col= e_col, pipe= pipe):
    
    embed_mat = np.array([x for x in predictions[e_col]])
# calculate distance between every embedding pair
    sim_mat = cosine_similarity(embed_mat,embed_mat)
#get sim score for a given sentence at position df.iloc[sentence_id]
    print("Similarities for Sentence : " + df.iloc[sent_iloc].Abstract)
    df['sim_score'] = sim_mat[sent_iloc]
    return df

sentence_to_compare= 12
sim_df_for_one_sent = get_sim_df_for_iloc(sentence_to_compare, predictions, e_col)
sim_df_for_one_sent.sort_values('sim_score', ascending=False)

# write sim scores to df

#sim_df.sort_values('sim_score', ascending = False)

Similarities for Sentence :  machine learning systems become democratized   becomes increasingly important  help users easily debug  models  however  current data tools  still primitive   comes  helping users trace model performance problems   way   data   focus   particular problem  slicing data  identify subsets   validation data   model performs poorly     important problem  model validation   overall model performance  fail  reflect    smaller subsets   slicing allows users  analyze  model performance    granular level  unlike general techniques  e g   clustering    find arbitrary slices   goal   find interpretable slices    easier  take action compared  arbitrary subsets    problematic  large   propose mathsf slice finder slicefinder     interactive framework  identifying  slices using statistical techniques  applications include diagnosing model fairness  fraud detection   identifying slices   interpretable  humans  crucial   research  part   larger trend  big data  artificial in

,Abstract,sim_score
12,machine learning systems become democratized ...,1.000000
13,recent years machine learning ml systems ...,0.959530
8,advent incorporation technology businesses...,0.939237
2,various methods machine learning implemente...,0.938906
7,artificial intelligence finding way ever a...,0.936523
16,machine learning community become alert wa...,0.932277
14,machine learning models play important role ...,0.931571
27,artificial intelligence ai based robots bec...,0.930172
0,several fundamental problems arise optimizat...,0.923994
15,application ai finance increasingly depend...,0.920368
